In [1]:
import pandas as pd
from ast import literal_eval

In [2]:
df = pd.read_csv("aromadb_combined.csv")
file_df = pd.read_csv("../file_cleaning_features.csv")

aroma_full_columns = literal_eval(file_df[file_df['dataset'] == 'aromadb']['new_label_columns'].to_list()[0])
df['odor'] = df['no aroma'].apply(lambda row: 1 if row == 0 else 0)

In [3]:
aroma_odor = df.copy()
aroma_odor.drop(columns=aroma_full_columns, inplace=True)
aroma_odor.to_csv("../aromadb_odor/aromadb_odor_combined.csv", index=False)
aroma_odor_dict = {"dataset": "aromadb_odor",
                    "unclean": None,
                    "label_columns": None,
                    "task": "binary",
                    "new_label_columns": str(["odor"]),
                    "task_dim": 1,
                    "n_datapoints": len(aroma_odor),
                    "features": str(['IsomericSMILES']),
                    "validate": True}


In [4]:
aroma_descriptor = df.copy()
aroma_descriptor = aroma_descriptor[aroma_descriptor['odor'] == 1]
aroma_descriptor.drop(columns=['no aroma', 'odor'], inplace=True)
aroma_descriptor.to_csv("../aromadb_descriptor/aromadb_descriptor_combined.csv", index=False)

aroma_descriptor_columns = aroma_full_columns
aroma_descriptor_columns.remove("no aroma")
aroma_descriptor_dict = {"dataset": "aromadb_descriptor",
                    "unclean": None,
                    "label_columns": None,
                    "task": "binary",
                    "new_label_columns": str(aroma_descriptor_columns),
                    "task_dim": 1,
                    "n_datapoints": len(aroma_descriptor),
                    "features": str(['IsomericSMILES']),
                    "validate": True}

In [5]:
file_df.drop(file_df[file_df['dataset'] == 'aromadb'].index, inplace=True)
new_aroma_df = pd.DataFrame([aroma_odor_dict, aroma_descriptor_dict])
file_df = pd.concat([file_df, new_aroma_df])

In [6]:
file_df.to_csv("../file_cleaning_features.csv",index=False)